## Make Allan Deviation Plot

In [1]:
import os
import obspy as obs
import matplotlib.pyplot as plt
import numpy as np
import allantools

from andbro__querrySeismoData import __querrySeismoData
from andbro__read_sds import __read_sds

In [2]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'

## Configurations

In [3]:
## Configurations

config = {}

config['tbeg'] = obs.UTCDateTime("2023-09-19 09:00")
config['tend'] = obs.UTCDateTime("2023-09-19 10:00")

config['seed'] = "BW.ROMY.10.BJZ"

config['repository'] = "archive"

config['output_figures'] = data_path+""

config['net'], config['sta'], config['loc'], config['cha'] = config['seed'].split(".")


## Load Data

### import raw sagnac data

In [71]:
if config['datatype'] == "raw":

    st = __read_sds(archive_path+"romy_archive", "BW.DROMY..FJZ", config['tbeg'], config['tend'])

    for tr in st:
        tr.data = tr.data*0.59604645e-6 # V / count  [0.59604645ug  from obsidian]
st

1 Trace(s) in Stream:
BW.DROMY..FJZ | 2023-09-19T09:00:00.000000Z - 2023-09-19T10:00:00.000000Z | 5000.0 Hz, 18000001 samples

In [68]:
def __instantaneous_frequency(st0, df, f_expected):

    from numpy import unwrap, diff, angle, pi, nanmean, sqrt, gradient
    from scipy.signal import hilbert

    ## prepare data
    # st0 = st0.detrend("demean")
    # st0 = st0.taper(0.01)
    # st0 = st0.filter("bandpass", freqmin=f_expected-50, freqmax=f_expected+50, corners=8, zerophase=True)

    # estimate instantaneous frequency
    analytic_signal = hilbert(st[0].data)
    amplitude_envelope = abs(analytic_signal)
    instantaneous_phase = unwrap(angle(analytic_signal))
    instantaneous_frequency = (diff(instantaneous_phase) / (2.0*pi) * df)

#     x = st[0].data
#     H = unwrap(angle(hilbert(x)))
#     # dH = diff(H, prepend=H[0])
#     # dx = diff(x, prepend=x[0])
#     dH = gradient(H)
#     dx = gradient(x)

#     instantaneous_frequency = ( dH * x - dx * H ) / (2 * pi * sqrt( x**2 + H**2 ) )


    print(instantaneous_frequency)


    ## cut first and last 5% (corrupted)
    dd = int(0.1*len(instantaneous_frequency))
    insta_f_cut = instantaneous_frequency[dd:-dd]

    ## time
    t = st0[0].times()
    t_mid = t[int((len(t))/2)]

    ## average
    # insta_f_cut_mean = nanmean(insta_f_cut)
    insta_f_cut_mean = nanmean(insta_f_cut)

    return t, insta_f_cut_mean

In [57]:
def __get_time_intervals(tbeg, tend, interval_seconds, interval_overlap):

    from obspy import UTCDateTime

    tbeg, tend = UTCDateTime(tbeg), UTCDateTime(tend)

    times = []
    t1, t2 = tbeg, tbeg + interval_seconds
    while t2 <= tend:
        times.append((t1, t2))
        t1 = t1 + interval_seconds - interval_overlap
        t2 = t2 + interval_seconds - interval_overlap

    return times

In [69]:
interval_seconds = 600
interval_overlap = 0

t = np.arange(0, int(config['tend'] - config['tbeg']), interval_seconds)

times = __get_time_intervals(config['tbeg'], config['tend'], interval_seconds, interval_overlap)

In [70]:
f_insta = []

for t1, t2 in times:
    _st = st.copy().trim(t1, t2, nearest_sample=False)

    _t, _f = __instantaneous_frequency(_st, _st[0].stats.sampling_rate, 553.5)

    print(_f)

    f_insta.append(_f)

[  -26.1679527    314.39446152   215.34538028 ... -1531.36489427
    57.21231184   485.86366197]
6.388550266803713e-05
[  -26.1679527    314.39446152   215.34538028 ... -1531.36489427
    57.21231184   485.86366197]
6.388550266803713e-05
[  -26.1679527    314.39446152   215.34538028 ... -1531.36489427
    57.21231184   485.86366197]
6.388550266803713e-05
[  -26.1679527    314.39446152   215.34538028 ... -1531.36489427
    57.21231184   485.86366197]
6.388550266803713e-05
[  -26.1679527    314.39446152   215.34538028 ... -1531.36489427
    57.21231184   485.86366197]
6.388550266803713e-05
[  -26.1679527    314.39446152   215.34538028 ... -1531.36489427
    57.21231184   485.86366197]
6.388550266803713e-05


In [ ]:
plt.plot(f_insta)